# Covid-19 Dashboard

The following is a simple dashboard that will be displayed using the [voila](https://voila.readthedocs.io/en/stable/index.html) Python dashboarding tool that converts notebooks to standalone dashboards. The ```voila``` package must be installed using *pip* or *conda* in order to run the dashboard.

The dashboard downloads and displays Covid-19 data from the Public Health England (PHE) website https://coronavirus.data.gov.uk.

In [31]:
import ipywidgets as wdg
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from datetime import datetime
from uk_covid19 import Cov19API

In [32]:
# an iPython  "magic" that enables the embedding of matplotlib output
%matplotlib inline
# make figures larger
plt.rcParams['figure.dpi'] = 100

In [33]:
# Load JSON files and store the raw data in some variable. Edit as appropriate
with open("tests_ts.json", "rt") as INFILE:
    init_data = json.load(INFILE)

In [34]:
def parse_date(datestring):
        """ Convert a date string into a pandas datetime object """
        return pd.to_datetime(datestring, format="%Y-%m-%d")

def wrangle_data(data):
    '''Takes raw JSON data and wrangles it into a dataframe'''
    # Putting the wrangling code into a function allows us to call it again after refreshing the data through 
    # the API.

    # Separates out the data from its container    
    datalist = data['data']

    # Extracts the dates and sort alphabetically
    dates=[dictionary['date'] for dictionary in datalist]
    dates.sort()

    # Finds the start and end dates of the data. parse_date converts dates to pandas format
    startdate = parse_date(dates[0])
    enddate = parse_date(dates[-1])

    # The date range becomes the index for the data frame, this fills in any missing dates
    # The data frame is then created using the index and the column names
    index = pd.date_range(startdate, enddate, freq = 'D')
    tests_df = pd.DataFrame(index = index, columns = ['cases', 'tests', 'testCap'])

    # Adding data to the data frame
    # Each entry is a dictionary with date, cases, tests, and testCap
    for entry in datalist: 
        date = parse_date(entry['date'])
        for column in ['cases', 'tests', 'testCap']:
            # Checks that nothing is there yet - just in case some dates are duplicated
            if pd.isna(tests_df.loc[date, column]): 
                # replace None with 0 in our data 
                value = float(entry[column]) if entry[column] != None else 0.0
                # .loc is used to access each location in the dataframe
                tests_df.loc[date, column] = value

    # .fillna is used to fill in any remaining gaps in data caused by missing dates
    tests_df.fillna(0.0, inplace=True)      
    
    return tests_df

# The wrangle function is called directly on the JSON data when the dashboard starts
tests_df = wrangle_data(init_data) # tests_df is the dataframe for plotting

## Development

This dashboard includes "canned" data in a ```.json``` file that is loaded upon starting the dashboard. This raw data is wrangled into a ```DataFrame``` that is then used for plotting. 

The dataset can be "refreshed" in order to update the DataFrame with up-to-date data. The Refresh button below will:
* Call the code that accesses the Public Health England API and download some fresh raw data.
* Wrangle that data into a dataframe and update the DataFrame for plotting.
* Force a redraw of the graph and indicate when the data was last updated.
* Alert the user if API access is unavailable.

The graph below features interactive controls that allow the user select which variables they wish to display on the graph. Ctrl-clicking allows multiple values to be displayed for comparison. Radio buttons allow the user to switch betweeen linear and logarithmic representation.

## Comparison of Cases, Tests, and Planned Test Capacity per Day Across the UK

* **Cases** refers to the number of ```new cases by publish date```.
* **Tests** refers to the number of ```new tests by publish date```.
* **TestCap** refers to the ```planned test capacity by publish date```.

The latest data can be seen by refreshing the data using the button below.

The graph clearly shows the intial spike in cases that occurred in March before the initial national lockdown, and the second rise that occurred in October. The number of cases can be seen to drop off towards the end of November, likely as a result of the reinstated lockdown measures. One particular point of note is the short dip and subsequent spike in cases at the end of September and Beginning of October. This could well be related to a PHE technical issue that resulted in cases between 25 September and 2 October not being included in the reported daily COVID-19 figures.

The graph shows that the number of tests has risen steadlily since March, and this increased testing will have contributed to the higher number of confirmed cases nationwide that is seen in the second spike. While the number of tests has risen, the number of tests can be seen to have consistently fallen short of the planned test capacity. The number of new tests fluctuates throughout each week due to the way the numbers are published, but, as of the end of November, it appears as though the number of daily tests is beginning to plateau.  

In [35]:
# API access code will be called by the button_callback function
def access_api():
    """ Accesses the PHE API. Returns raw data in the same format as data loaded from the "canned" JSON file. """
    filters = [
    'areaType=overview'
    ]

    # These are the PHE metrics we wish to plot
    structure = {
        "date": "date",
        "cases": "newCasesByPublishDate",
        "tests": "newTestsByPublishDate",
        "testCap": "plannedCapacityByPublishDate"
    }

    api = Cov19API(filters = filters, structure=structure)

    # Calls the server and fetches the data
    tests_ts = api.get_json()

    # Saves the time series in a JSON file
    #with open("tests_ts.json", "wt") as OUTF:
        #json.dump(tests_ts, OUTF)
        
    return tests_ts # return data read from the API

In [36]:
def api_button_callback(button):
    """ Button callback - Accesses API, wrangles data, updates global variable df 
    used for plotting, outputs the refresh time, or an error message if unsuccessful"""
    # Fetches fresh data from the PHE API
    try:
        apidata = access_api()
        # Wrangle the data and overwrite the dataframe for plotting
        global tests_df
        tests_df = wrangle_data(apidata)

        # The graph won't refresh until the user interacts with the widget 
        # This function simulates the interaction, by flipping the scale value back and forth
        refresh_graph()

        # The button icon changes to "check" sign to indicate that the data has been refreshed
        apibutton.icon = "check"
        # Button can be disabled to prevent repeated calls to the API
        # apibutton.disabled = True

        # Prints the date and time of the last data refresh
        # output widget is cleared before outputting the formatted datetime
        now = datetime.now()
        date_time = now.strftime("%m/%d/%Y, %H:%M:%S")
        with out:
            out.clear_output()
            print("Data last refreshed at:", date_time)
    
    # If the data refresh fails, output to the user to avoid the dashboard crashing
    except:
        with out:
            out.clear_output()
            print("Data Refresh Failed; Public Health England API may be unavailable")
        
# Creates the button widget
apibutton = wdg.Button(
    description = 'Refresh data',
    disabled = False,
    button_style = 'info', # 'success', 'info', 'warning', 'danger' or ''
    tooltip = 'Click to download current Public Health England data',
    icon = 'download' # (FontAwesome names without the `fa-` prefix)
)


# Registers the callback function with the button and displays it
apibutton.on_click(api_button_callback)

display(apibutton)

# This is an output widget to display the last refresh time to the user
out = wdg.Output()
display(out)

Button(button_style='info', description='Refresh data', icon='download', style=ButtonStyle(), tooltip='Click t…

Output()

In [37]:
# Creates controls for displaying different data
series = wdg.SelectMultiple(
    options = ['cases', 'tests', 'testCap'],
    value = ['cases', 'tests', 'testCap'],
    rows = 3,
    description = 'Stats:',
    disabled = False
)

# Creates controls for changing the graph scale
scale = wdg.RadioButtons(
    options = ['linear', 'log'],
    value = 'linear', # Defaults to 'linear'
#   layout={'width': 'max-content'}, # If the items' names are long
    description = 'Scale:',
    disabled = False
)

# Creates a container for the controls
controls = wdg.HBox([series, scale])

def timeseries_graph(gcols, gscale):
    if gscale == 'linear':
        logscale = False
    else:
        logscale = True
    ncols = len(gcols)
    if ncols > 0:
        tests_df[list(gcols)].plot(logy = logscale)
    else:
        print("Click to select data for graph")
        print("(CTRL-Click to select more than one category)")
        
# This function changes the scale value back and forth to force the interactive graph to be redrawn
def refresh_graph():
    """ We change the value of the widget in order to force a redraw of the graph;
    used when the data has been updated"""
    current = scale.value
    if current == scale.options[0]:
        other = scale.options[1]
    else:
        other = scale.options[0]
    scale.value = other # changes the value to force a redraw
    scale.value = current # this changes it straight back

# keep calling timeseries_graph(gcols=value_of_series, gscale=value_of_scale); capture output in variable graph   
graph = wdg.interactive_output(timeseries_graph, {'gcols': series, 'gscale': scale})

display(controls, graph)

Output()

*Copyright &copy; 2020, Joseph Cook (joe.ri.cook@outlook.com). All Rights Reserved.* <br/>
*Based on UK Government [data](https://coronavirus.data.gov.uk/) published by [Public Health England](https://www.gov.uk/government/organisations/public-health-england).*